In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import sys
from selenium.webdriver.chrome.options import Options
import time
import requests
import pandas as pd

In [ ]:
pip install selenium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#---------  ADD START_PAGE AND END_PAGE------------#
start_page = 1
end_page = 2

In [ ]:

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
driver.get('https://datawarehouse.dbd.go.th/index')


time.sleep(2)
button = driver.find_element(By.ID, "btnWarning").send_keys("\n")

time.sleep(2)
button = driver.find_element(By.CLASS_NAME, "card-link").send_keys("\n")

time.sleep(2)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "searchicon"))).click()

time.sleep(2)
search_input = driver.find_element(By.ID,"cPage")
search_input.clear()
search_input.send_keys(start_page)
search_input.send_keys(Keys.ENTER)
print('click')

time.sleep(10)

for i in range(start_page,end_page+1):
    time.sleep(2)
    if i== start_page:
        table = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "fixTable"))).get_attribute("outerHTML") #find table content
        df = pd.read_html(table)
        df = df[0]
        tr = driver.find_elements(By.CSS_SELECTOR, "#fixTable > tbody > tr") # find tr data-href
        links =[link.get_attribute('data-href') for link in tr]
        df['links']= links
        df['page'] = i
    else:
        next_button  = driver.find_element(By.ID, "next").send_keys("\n")
        time.sleep(10)
        table = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "fixTable"))).get_attribute("outerHTML") #find table content
        temp_df = pd.read_html(table)
        temp_df = temp_df[0]
        tr = driver.find_elements(By.CSS_SELECTOR, "#fixTable > tbody > tr") # find tr data-href
        links =[link.get_attribute('data-href') for link in tr]
        temp_df['links']= links
        temp_df['page'] = i
        df = pd.concat([df,temp_df],ignore_index=True)

    df.to_csv(f"/content/drive/MyDrive/op_dbd/page_{start_page}_to_{end_page}.csv")
    print(f"done concat page{i}")

driver.close()